In [ ]:
from pennylane import numpy as np
import pennylane as qml
from pennylane import qchem
from matplotlib import pyplot as plt
from functions_vqe_cnot import vqe_uccsd, vqe_hee, vqe_ng
import numpy as np
from joblib import Parallel, delayed

device = 'cpu'

## He<sub>2</sub> hamiltonian definition 

In [2]:
dataset = qml.data.load('qchem', molname="He2")[0]
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  8
The Hamiltonian is  -1.6743100655776797 * I([0, 1, 2, 3, 4, 5, 6, 7]) + 0.3416412951109973 * Z(0) + 0.01523407338776362 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.01523407338776362 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.3416408311620236 * Z(2) + 0.025441159889832372 * (Z(0) @ Z(2)) + 0.015234388911867857 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.015234388911867857 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 7.317214998671384e-08 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 7.317214998671384e-08 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.5193693204313432 * Z(4) + 0.09152839641299164 * (Z(0) @ Z(4)) + -0.5193733761290926 * Z(6) + 0.0915284611478574 * (Z(0) @ Z(6)) + 0.34164129511099733 * Z(1) + 0.14108387203893755 * (Z(0) @ Z(1)) + 0.03956129782205163 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.03956129782205163 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.11564281616296 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.11564281616296 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.11564281616296 * (X(0) @ X(1) @

In [3]:
dev = qml.device("lightning.qubit", wires=qubits)

In [4]:
electrons = 4
hf_state = qml.qchem.hf_state(electrons, qubits)
singles, doubles = qchem.excitations(electrons, qubits)
print(hf_state)

[1 1 1 1 0 0 0 0]


In [5]:
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

In [6]:
max_iterations = 50
conv_tol = 1e-06

opt = qml.GradientDescentOptimizer(stepsize=0.1)

## Distribution of results

In [62]:
def run_single_simulation(i, l, j, function):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = function(H, qubits, l, opt, max_iterations=20, conv_tol=1e-06, verbose=False)
    
    return i, j, (energy, grad_norm, grad_var, conv)

In [89]:
max_iterations = 20
layers = np.arange(2, 13, 2)
n_layers = len(layers)
n_runs = 5

In [76]:
tasks = (delayed(run_single_simulation)(i, l, j, vqe_ng)
         for i, l in enumerate(layers)
         for j in range(n_runs))

results = Parallel(n_jobs=-1, verbose=11)(tasks)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done   5 out of  25 | elapsed:    4.8s remaining:   19.4s
[Parallel(n_jobs=-1)]: Done   8 out of  25 | elapsed:    5.6s remaining:   12.0s
[Parallel(n_jobs=-1)]: Done  11 out of  25 | elapsed:    7.6s remaining:    9.6s
[Parallel(n_jobs=-1)]: Done  14 out of  25 | elapsed:   11.2s remaining:    8.8s
[Parallel(n_jobs=-1)]: Done  17 out of  25 | elapsed:   13.1s remaining:    6.2s
[Parallel(n_jobs=-1)]: Done  20 out of  25 | elapsed:   15.1s remaining:    3.8s
[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:   17.1s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   18.0s finished


In [94]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_layers, n_runs, max_iterations+1))
grad_norms = np.zeros((n_layers, n_runs, max_iterations))
grad_variances = np.zeros((n_layers, n_runs, max_iterations))
convergences = np.zeros((n_layers, n_runs), dtype=bool)

for i, j, result in results:
    energies[i, j, :] = np.array(result[0])
    grad_norms[i, j, :] = np.array(result[1])
    grad_variances[i, j, :] = np.array(result[2])
    convergences[i, j] = result[3]